In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,Bidirectional
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [3]:
# Check if GPU is available
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("GPU not found. Please check your Kaggle settings.")

Default GPU Device: /device:GPU:0


In [4]:
df = pd.read_csv("/kaggle/input/dataset-merged/sih_fec_training_data_merged.csv")

In [5]:
df.shape

(407400, 2)

In [6]:
feed = pd.DataFrame()
target = pd.DataFrame()

In [7]:
feed['encoded_data_string'] = df['encoded_data_string']
target['encoding'] = df['encoding']

In [8]:
X = feed  # Features (all columns in the DataFrame)
y = target  # Target variable (if applicable)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
data = []
for bit_string in X_train['encoded_data_string']:
    bit_array = np.array(list(bit_string), dtype=int)
    data.append(bit_array)

In [11]:
Xtrain_array = np.reshape(data, (325920, 2048, 1))

In [12]:
Xtrain_array.shape

(325920, 2048, 1)

In [13]:
data1 = []
for bit_string in X_test['encoded_data_string']:
    bit_array = np.array(list(bit_string), dtype=int)
    data1.append(bit_array)

In [14]:
Xtest_array = np.reshape(data1, (81480, 2048, 1))

In [15]:
ytrain_array = y_train['encoding'].values
ytest_array = y_test['encoding'].values

In [16]:
# np.save('Xtrain_array.txt', Xtrain_array)

In [17]:
ytrain_mapped = ytrain_array - 1

In [18]:
np.unique(ytrain_array)

array([1, 2, 3, 4])

In [19]:
# np.save('ytrain_array.txt', ytrain_array)

In [20]:
# np.save('Xtest_array.txt', Xtest_array)

In [21]:
# np.save('ytest_array.txt', ytest_array)

In [22]:
Xtrain_array_gpu = tf.convert_to_tensor(Xtrain_array, dtype=tf.float32)
ytrain_mapped_gpu = tf.convert_to_tensor(ytrain_mapped, dtype=tf.int32)

In [23]:
input_shape = (2048, 1)

# Assuming Xtrain_array_gpu and ytrain_mapped_gpu are TensorFlow tensors
train_dataset = tf.data.Dataset.from_tensor_slices((Xtrain_array_gpu, ytrain_mapped_gpu)).shuffle(buffer_size=2000).batch(133)
test_dataset = tf.data.Dataset.from_tensor_slices((Xtest_array, y_test)).batch(133)


In [24]:
# Define the model within the strategy scope
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 2


In [25]:
with strategy.scope():
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [26]:
model.load_weights('/kaggle/input/weight97a6e/current_model_weights97a6e.h5')

In [27]:
# Add ModelCheckpoint callback
model_checkpoint = ModelCheckpoint(filepath='model_checkpoint.h5', save_best_only=True, period=3)# Train the model using the distributed dataset
early_stopping = EarlyStopping(monitor='val_loss', patience=200, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.0002, patience=3, min_lr=1e-6)

model.fit(train_dataset, epochs=200, validation_data=test_dataset,batch_size=133,callbacks=[model_checkpoint,early_stopping,reduce_lr])

Epoch 1/200
   9/2451 [..............................] - ETA: 22:23 - loss: 0.0873 - accuracy: 0.9624

KeyboardInterrupt: 

In [30]:
model.save_weights('current_model_weights98a14e.h5')

In [28]:
np.unique(ytest_array)

array([1, 2, 3, 4])

In [29]:
ytest_mapped = ytest_array - 1

In [30]:
np.unique(ytest_mapped)

array([0, 1, 2, 3])

In [31]:
ytest_mapped

array([2, 0, 1, ..., 2, 3, 2])

In [32]:

if np.isnan(ytest_mapped).any() or np.isinf(ytest_mapped).any():
    print("True labels contain NaN or infinite values.")
    
else:
    print("np")


np


In [33]:
ytest_mapped.shape

(81480,)

In [34]:
ytest_array.shape

(81480,)

In [35]:
Xtest_array_gpu = tf.convert_to_tensor(Xtest_array, dtype=tf.float32)
ytest_mapped_gpu = tf.convert_to_tensor(ytest_mapped, dtype=tf.int32)

In [36]:
loss, accuracy = model.evaluate(Xtest_array_gpu, ytest_mapped_gpu)

print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy * 100:.2f}%")

  75/2547 [..............................] - ETA: 5:55 - loss: 0.0748 - accuracy: 0.9733

KeyboardInterrupt: 

In [39]:
# Save the model as a pickle file
with open('sih_2048_snr2.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)